In [1]:
from data_loader import Data_loader
import numpy as np
import pickle as pkl

labeled_tids = np.loadtxt('../data/labeled_tids.np', dtype='int')

option = 'word'
dl = Data_loader(option=option, labeled_only=True)
tweets = [dl[t] for t in labeled_tids]

Data loader ...
Loading vocabulary ...
40000 vocab is considered.
Loading tweets ...
Processing tweets ...
Data loader initialization finishes


In [12]:
time = [d['created_at'] for d in tweets]

def time2bin(dt):
    return (dt.year, dt.month)

time = [time2bin(t) for t in time]
count = {}
for t in time:
    if count.get(t) is None:
        count[t] = 0
    count[t] += 1

sorted_keys = sorted(count.keys())
for key in sorted_keys:
    print(str(key) + ': ' + str(count[key]))

(2014, 1): 102
(2014, 3): 357
(2014, 4): 357
(2017, 5): 251
(2017, 6): 588
(2017, 7): 600
(2017, 10): 53
(2017, 11): 3496
(2017, 12): 3097
(2018, 1): 1872


In [13]:
def d2c(d):
    i = np.eye(3)
    time = d['created_at']
    if time.year == 2014:
        return 0
    elif time.year == 2017 and time.month < 10:
        return 1
    else:
        return 2
    
bins = [sorted([d for d in tweets if d2c(d) == bin_idx], key=lambda d: d['created_at']) 
        for bin_idx in range(3)]
print([len(b) for b in bins])

for b in bins:
    for _ in range(len(b)):
        b[_]['offset_time'] = (b[_]['created_at'] - b[0]['created_at']).total_seconds()


[816, 1439, 8518]


In [19]:
for b in bins:
    for _ in range(len(b)):
        b[_]['time_proportion'] = (b[_]['offset_time'] - b[0]['offset_time']) / (b[-1]['offset_time'] - b[0]['offset_time'])


In [24]:
for bin_idx in range(3):
    b = bins[bin_idx]
    for tweet in b:
        t_feat = np.zeros(6)
        t_feat[bin_idx] = 1
        t_feat[bin_idx + 3] = tweet['time_proportion']
        tweet['time_feature'] = t_feat

In [28]:
id2time_feat = dict([(tweet['tweet_id'], tweet['time_feature']) for tweet in tweets])

In [30]:
pkl.dump(id2time_feat, open('../data/id2time_feat.pkl', 'wb'))

In [10]:
user_pkl = pkl.load(open('../model/non_anonymized_user.pkl','rb'))
id2user_property = dict([(user_pkl[u]['id'], user_pkl[u]) for u in user_pkl])
print(id2user_property)

{'unlabeled_user_rt': 56, 'id': 1200, 'labeled_user_post': 0, 'occurence_in_labeled': 1, 'unlabeled_user_mentioned': 53, 'unlabeled_user_post': 0, 'labeled_user_rt': 0, 'labeled_user_mentioned': 1, 'occurence_in_unlabeled': 109}
